<a href="https://colab.research.google.com/github/EnsarIshakoglu/NLP/blob/topic-modelling/NLP_Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [186]:
# Source: https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#1introduction
# Run in python console
import nltk; nltk.download('stopwords')
import zipfile
import os

# Run in terminal or command prompt
!python3 -m spacy download en

# SKlearn
# from sklearn.model_selection import train_test_split
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import KFold
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.preprocessing import LabelEncoder
# from sklearn.feature_extraction.text import CountVectorizer
# from imblearn.over_sampling import SMOTE

# Data processing
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pandas as pd
import numpy as np

# Stackapi to fetch stackoverflow api
!pip install stackapi
from stackapi import StackAPI

from pprint import pprint

# Gensim
!pip install gensim==3.8.3
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
     |████████████████████████████████| 12.0 MB 5.1 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


### Mount colab drive to google drive

In [187]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Fetch data

In [188]:
# Get the data from stackoverflow sorted by votes
def fetch_data():
  if not exists('/content/NLP-question-data'):
    !git clone https://github.com/EnsarIshakoglu/NLP-question-data.git
  
  data = pd.read_csv('/content/NLP-question-data/questions.csv')
  
  return data

## Remove useless data

In [189]:
def clean_data(df):  
  df = df[['tags', 'body']]

  # Strip html tags with regex:
  df['body'] = df['body'].str.replace(r'<[^<>]*>', '', regex=True)

  # Get first tag for multi-class classification
  row_count = df.shape[0]

  # for i in range(row_count):
  #   df['tags'].iloc[i] = df['tags'].iloc[i][0]
  
  return df

### Create folder and file from df, unmout drive after

In [190]:
from os.path import exists

!mkdir stackoverflow

if not exists('/content/stackoverflow/questions.csv') or not exists('/content/NLP-question-data'):
  df = fetch_data()[['tags', 'body']]
  df = clean_data(df)
  df.to_csv('/content/stackoverflow/questions.csv')
  print("Fetched data from stackoverflow, removed the useless data and saved it in stackoverflow/questions.csv")

drive.flush_and_unmount()
df

mkdir: cannot create directory ‘stackoverflow’: File exists


,Unnamed: 0,tags,body
0,0,"['java', 'c++', 'performance', 'cpu-architectu...",Here is a piece of C++ code that shows some ve...
1,1,"['git', 'version-control', 'git-commit', 'undo']",I accidentally committed the wrong files to Gi...
2,2,"['git', 'version-control', 'git-branch', 'git-...",I want to delete a branch both locally and rem...
3,3,"['git', 'version-control', 'git-pull', 'git-fe...",What are the differences between git pull and ...
4,4,"['python', 'iterator', 'generator', 'yield', '...",What is the use of the yield keyword in Python...
...,...,...,...
17495,17495,"['python', 'pandas', 'dataframe', 'nan']",I have a Pandas Dataframe as below:\n itm...
17496,17496,"['java', 'datetime', 'java-8', 'java-time']",What is the best way to convert a java.util.Da...
17497,17497,['visual-studio-code'],Is there a Compare feature like the Plugin for...
17498,17498,"['visual-studio-code', 'whitespace', 'vscode-s...","Is it possible to show whitespace characters, ..."


### Load file from drive

In [191]:
df = pd.read_csv('/content/stackoverflow/questions.csv')

df.head()

,Unnamed: 0,tags,body
0,0,"['java', 'c++', 'performance', 'cpu-architectu...",Here is a piece of C++ code that shows some ve...
1,1,"['git', 'version-control', 'git-commit', 'undo']",I accidentally committed the wrong files to Gi...
2,2,"['git', 'version-control', 'git-branch', 'git-...",I want to delete a branch both locally and rem...
3,3,"['git', 'version-control', 'git-pull', 'git-fe...",What are the differences between git pull and ...
4,4,"['python', 'iterator', 'generator', 'yield', '...",What is the use of the yield keyword in Python...


In [192]:
print(f"There are {len(df['body'].unique())} rows in the dataset.")

There are 3900 rows in the dataset.


In [193]:
df = df[~df['body'].duplicated()]
print(f"There are {len(df)} rows in the deduplicated dataset.")

There are 3900 rows in the deduplicated dataset.


## Topic modeling

In [194]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

## Prepare data

In [195]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(df['body']))

print(data_words[:1])

[['here', 'is', 'piece', 'of', 'code', 'that', 'shows', 'some', 'very', 'peculiar', 'behavior', 'for', 'some', 'strange', 'reason', 'sorting', 'the', 'data', 'before', 'the', 'timed', 'region', 'miraculously', 'makes', 'the', 'loop', 'almost', 'six', 'times', 'faster', 'include', 'lt', 'algorithm', 'gt', 'include', 'lt', 'ctime', 'gt', 'include', 'lt', 'iostream', 'gt', 'int', 'main', 'generate', 'data', 'const', 'unsigned', 'arraysize', 'int', 'data', 'arraysize', 'for', 'unsigned', 'lt', 'arraysize', 'data', 'std', 'rand', 'with', 'this', 'the', 'next', 'loop', 'runs', 'faster', 'std', 'sort', 'data', 'data', 'arraysize', 'test', 'clock_t', 'start', 'clock', 'long', 'long', 'sum', 'for', 'unsigned', 'lt', 'for', 'unsigned', 'lt', 'arraysize', 'primary', 'loop', 'if', 'data', 'gt', 'sum', 'data', 'double', 'elapsedtime', 'static_cast', 'lt', 'double', 'gt', 'clock', 'start', 'clocks_per_sec', 'std', 'cout', 'lt', 'lt', 'elapsedtime', 'lt', 'lt', 'std', 'cout', 'lt', 'lt', 'quot', 'sum

In [196]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['here', 'is', 'piece', 'of', 'code', 'that', 'shows', 'some', 'very', 'peculiar', 'behavior', 'for', 'some', 'strange', 'reason', 'sorting', 'the', 'data', 'before', 'the', 'timed', 'region', 'miraculously', 'makes', 'the', 'loop', 'almost', 'six', 'times', 'faster', 'include', 'lt', 'algorithm', 'gt', 'include', 'lt', 'ctime', 'gt', 'include', 'lt', 'iostream', 'gt', 'int', 'main', 'generate', 'data', 'const', 'unsigned', 'arraysize', 'int', 'data', 'arraysize', 'for', 'unsigned', 'lt', 'arraysize', 'data', 'std', 'rand', 'with', 'this', 'the', 'next', 'loop', 'runs', 'faster', 'std', 'sort', 'data', 'data', 'arraysize', 'test', 'clock_t', 'start', 'clock', 'long', 'long', 'sum', 'for', 'unsigned', 'lt', 'for', 'unsigned', 'lt', 'arraysize', 'primary', 'loop', 'if', 'data', 'gt', 'sum', 'data', 'double', 'elapsedtime', 'static_cast', 'lt', 'double', 'gt', 'clock', 'start', 'clocks_per_sec', 'std_cout', 'lt', 'lt', 'elapsedtime', 'lt', 'lt', 'std_cout', 'lt', 'lt', 'quot', 'sum', 'quo

In [197]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [198]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'ADV'])

print(data_lemmatized[:1])

[['piece', 'code', 'show', 'peculiar', 'behavior', 'strange', 'reason', 'sort', 'datum', 'time', 'region', 'miraculously', 'make', 'loop', 'almost', 'time', 'faster', 'include', 'include', 'include', 'main', 'generate', 'datum', 'const', 'unsigne', 'datum', 'loop', 'run', 'fast', 'datum', 'datum', 'test', 'start', 'clock', 'long', 'long', 'sum', 'unsigne', 'datum', 'double', 'elapsedtime', 'lt', 'double', 'clock', 'start', 'datum', 'run', 'second', 'sort', 'code', 'run', 'second', 'sort', 'take', 'time', 'pass', 'array', 'actually', 'worth', 'need', 'calculate', 'unknown', 'array', 'initially', 'think', 'may', 'language', 'compiler', 'try', 'random', 'public', 'class', 'main', 'public', 'static', 'void', 'main', 'string', 'arg', 'generate', 'datum', 'int', 'arraysize', 'next', 'run', 'fast', 'test', 'long', 'start', 'system', 'nanotime', 'long', 'sum', 'lt', 'system', 'println', 'system', 'start', 'system', 'similar', 'less', 'extreme', 'result', 'first', 'thought', 'sort', 'bring', 'd

In [199]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 5), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 2), (11, 4), (12, 2), (13, 1), (14, 1), (15, 8), (16, 1), (17, 2), (18, 1), (19, 1), (20, 1), (21, 2), (22, 2), (23, 1), (24, 1), (25, 1), (26, 3), (27, 1), (28, 3), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 4), (36, 2), (37, 2), (38, 3), (39, 2), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 4), (55, 2), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 4), (62, 2), (63, 1), (64, 1), (65, 1), (66, 1), (67, 6), (68, 1), (69, 4), (70, 1), (71, 1), (72, 1), (73, 3), (74, 4), (75, 1), (76, 1), (77, 2), (78, 2), (79, 1), (80, 3), (81, 1), (82, 1), (83, 2), (84, 2), (85, 1), (86, 1)]]


In [200]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('actually', 1),
  ('almost', 1),
  ('amp', 1),
  ('arg', 1),
  ('array', 5),
  ('arraysize', 1),
  ('behavior', 1),
  ('bring', 1),
  ('calculate', 1),
  ('class', 1),
  ('clock', 2),
  ('code', 4),
  ('compiler', 2),
  ('const', 1),
  ('data', 1),
  ('datum', 8),
  ('different', 1),
  ('double', 2),
  ('effect', 1),
  ('elapsedtime', 1),
  ('extreme', 1),
  ('fast', 2),
  ('faster', 2),
  ('first', 1),
  ('flag', 1),
  ('followup', 1),
  ('generate', 3),
  ('go', 1),
  ('include', 3),
  ('independent', 1),
  ('initially', 1),
  ('int', 1),
  ('language', 1),
  ('late', 1),
  ('less', 1),
  ('long', 4),
  ('loop', 2),
  ('lt', 2),
  ('main', 3),
  ('make', 2),
  ('matter', 1),
  ('may', 1),
  ('miraculously', 1),
  ('modern', 1),
  ('nanotime', 1),
  ('need', 1),
  ('next', 1),
  ('optimization', 1),
  ('option', 1),
  ('order', 1),
  ('pass', 1),
  ('peculiar', 1),
  ('piece', 1),
  ('println', 1),
  ('processing', 4),
  ('public', 2),
  ('random', 1),
  ('reason', 1),
  ('region',

In [201]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [202]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.067*"know" + 0.047*"question" + 0.038*"first" + 0.036*"case" + '
  '0.034*"problem" + 0.030*"different" + 0.030*"understand" + 0.027*"may" + '
  '0.027*"point" + 0.023*"mean"'),
 (1,
  '0.065*"use" + 0.042*"way" + 0.040*"want" + 0.035*"try" + 0.035*"get" + '
  '0.033*"code" + 0.032*"work" + 0.028*"find" + 0.023*"new" + 0.023*"need"'),
 (2,
  '0.201*"quot" + 0.117*"public" + 0.096*"static" + 0.089*"main" + '
  '0.082*"date" + 0.077*"null" + 0.050*"throw" + 0.040*"compare" + '
  '0.029*"mode" + 0.027*"parse"'),
 (3,
  '0.101*"type" + 0.098*"text" + 0.054*"input" + 0.049*"page" + 0.047*"css" + '
  '0.045*"style" + 0.037*"base" + 0.033*"button" + 0.033*"form" + '
  '0.027*"browser"'),
 (4,
  '0.139*"would" + 0.071*"name" + 0.067*"value" + 0.066*"example" + '
  '0.034*"could" + 0.032*"check" + 0.031*"element" + 0.031*"possible" + '
  '0.029*"number" + 0.028*"contain"'),
 (5,
  '0.233*"image" + 0.087*"send" + 0.077*"search" + 0.074*"site" + 0.037*"rule" '
  '+ 0.036*"person" + 0.03

In [203]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -13.030529242972014

Coherence Score:  0.37221649141533986


In [204]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.325483  0.166686       1        1  28.572321
4     -0.211241 -0.346752       2        1  11.759688
0     -0.186552  0.244108       3        1   9.467061
14    -0.115246  0.023553       4        1   6.551493
17    -0.155954 -0.079579       5        1   6.182464
3     -0.045146 -0.081855       6        1   5.192719
8     -0.024128 -0.057262       7        1   4.278256
16    -0.016936  0.015573       8        1   4.140637
18     0.041264  0.002797       9        1   3.517070
7      0.049550  0.000971      10        1   2.958740
9      0.051432  0.001384      11        1   2.833288
6      0.070798  0.005127      12        1   2.401390
2      0.081047  0.007084      13        1   2.097822
15     0.078522  0.023155      14        1   1.890368
13     0.102198  0.012145      15        1   1.879490
11     0.112435  0.011908      16        1   1.664425
10     0.096311  0.009487      17        1   1.626453
5      0.112983  0.011733      18        1   1.424887
12     0.140643  0.014735      19        1   0.891372
19     0.143502  0.015000      20        1   0.670056, topic_info=           Term         Freq        Total Category  logprob  loglift
96         file  2234.000000  2234.000000  Default  30.0000  30.0000
266       would  2030.000000  2030.000000  Default  29.0000  29.0000
276    function   969.000000   969.000000  Default  28.0000  28.0000
72       string   877.000000   877.000000  Default  27.0000  27.0000
181         use  2308.000000  2308.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
2567    angular     0.037130     1.134574  Topic20 -10.0200   1.5860
1048     native     0.037124     1.134337  Topic20 -10.0202   1.5860
1680    install     0.037126     1.134487  Topic20 -10.0201   1.5859
1031  implement     0.037123     1.134360  Topic20 -10.0202   1.5860
5550     loader     0.037124     1.134430  Topic20 -10.0202   1.5859

[734 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
611       1  0.996827       able
414       8  0.994495     accept
980       7  0.993035     access
937       2  0.986114    achieve
612      12  0.994080     action
...     ...       ...        ...
107       5  0.993766      wrong
778       7  0.970367        xml
968      18  0.935359  yesterday
316       4  0.981654        yet
153      17  0.917666      yield

[512 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 5, 1, 15, 18, 4, 9, 17, 19, 8, 10, 7, 3, 16, 14, 12, 11, 6, 13, 20])

In [205]:
# Install java
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
  !java -version       #check java version
install_java()

openjdk version "11.0.11" 2021-04-20
OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04)
OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)


In [ ]:
# Install mallet
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
!unzip mallet-2.0.8.zip

--2021-12-07 13:11:31--  http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
Resolving mallet.cs.umass.edu (mallet.cs.umass.edu)... 128.119.246.70
Connecting to mallet.cs.umass.edu (mallet.cs.umass.edu)|128.119.246.70|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://mallet.cs.umass.edu/dist/mallet-2.0.8.zip [following]
--2021-12-07 13:11:31--  https://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
Connecting to mallet.cs.umass.edu (mallet.cs.umass.edu)|128.119.246.70|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16184794 (15M) [application/zip]
Saving to: ‘mallet-2.0.8.zip.1’

mallet-2.0.8.zip.1  100%[===================>]  15.43M  9.75MB/s    in 1.6s    

2021-12-07 13:11:33 (9.75 MB/s) - ‘mallet-2.0.8.zip.1’ saved [16184794/16184794]

Archive:  mallet-2.0.8.zip
replace mallet-2.0.8/bin/classifier2info? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
mallet_path = '/content/mallet-2.0.8/bin/mallet'

In [ ]:
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

In [ ]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=20, step=6)

In [ ]:
# Show graph
limit=20; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
# Select the model and print the topics
optimal_model = model_list[2]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

In [ ]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)

    return(sent_topics_df)

In [ ]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=texts)

In [ ]:
# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

In [ ]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.head()